In [1]:
import time
import pickle

import numpy as np

from helpers import *

In [2]:
# Dictionnary mapping the video_id to the channel_id
vid_to_channels = pd.read_pickle("/dlabdata1/youtube_large/id_to_channel_mapping.pickle")

In [3]:
occurent_users = dict_occurent_users()
dict_channel_ind, dict_ind_channel, channels_id = english_channels()

# -------------------------------------------------------------------------------------------

Compute the number of comments per channel and store it into the YouTube project

In [4]:
'''
Add a comment into the a dictionnary keeping track of the number of comments per channel
PARAMETERS:
    - dictionnary: dicionnary mapping a channel index with it's corresponding number of comments
    - corr_channel: the channel index in which we want to add a comment
'''
def add_comment(dictionnary, corr_channel):
    if corr_channel in dictionnary:
        dictionnary[corr_channel] += 1
    else:
        dictionnary[corr_channel] = 1

In [ ]:
# Adjust chunk_size as necessary -- defaults to 16,384 if not specific
reader = Zreader("/dlabdata1/youtube_large/youtube_comments.ndjson.zst", chunk_size=160384)

# parameters
idx = 1
comments_per_channels = {}
user = ''
begin_time = time.time()


# Read each line from the reader
for line in reader.readlines():
    line_split = line.replace('"', '').split(',')
    if len(line_split) == 9:
        author_id = line_split[0]
        if vid_to_channels.get(line_split[2]) in channels_id:
            corr_channel = dict_channel_ind[vid_to_channels[line_split[2]]]
            if author_id == user:
                if author_id in occurent_users:
                    if occurent_users[author_id] <= 1:
                        add_comment(comments_per_channels, corr_channel)
                else:
                    add_comment(comments_per_channels, corr_channel)
            else:
                if author_id in occurent_users:
                    occurent_users[author_id] += 1
                    if occurent_users[author_id] <= 1:
                        add_comment(comments_per_channels, corr_channel)
                else:
                    add_comment(comments_per_channels, corr_channel)
            
            user = author_id
    idx += 1
    if idx % 1000000000 == 0:
        print(begin_time-time.time())
        begin_time = time.time()
    

-5327.728192329407
-5531.879767417908
-5633.614473581314
-6021.908686161041


In [ ]:
# Store the comments_per_channel dictionnary into the YouTube project
with open("/dlabdata1/youtube_large/jouven/comments_per_channel.pkl",'wb') as f:
     pickle.dump(comments_per_channels, f)
f.close()